In [166]:
import nemo
import copy
import tqdm
import timeit
import pathlib
import attrdict
import numpy as np

from ruamel import yaml
from Bio import pairwise2
from nemo.collections import asr as nemo_asr
from nemo.collections.asr.helpers import post_process_predictions, post_process_transcripts, word_error_rate

In [167]:
runner = nemo.core.NeuralModuleFactory(
    placement=nemo.core.DeviceType.GPU
)

In [168]:
MODEL_CONFIG = 'examples/asr/configs/quartznet15x5.yaml'
yaml_loader = yaml.YAML(typ="safe")
with open(MODEL_CONFIG) as f:
    config = attrdict.AttrDict(yaml_loader.load(f))
labels = list(config.labels)
str(labels)

'[\' \', \'a\', \'b\', \'c\', \'d\', \'e\', \'f\', \'g\', \'h\', \'i\', \'j\', \'k\', \'l\', \'m\', \'n\', \'o\', \'p\', \'q\', \'r\', \'s\', \'t\', \'u\', \'v\', \'w\', \'x\', \'y\', \'z\', "\'"]'

In [169]:
EVAL_DATASETS = [
    '/home/stanislavv/data/nemo-librispeech/train_clean_100.json',
    '/home/stanislavv/data/nemo-librispeech/train_clean_360.json',
    '/home/stanislavv/data/nemo-librispeech/train_other_500.json',
]
eval_dl_params = copy.deepcopy(config.AudioToTextDataLayer)
eval_dl_params.update(config.AudioToTextDataLayer["train"])
del eval_dl_params["train"]
del eval_dl_params["eval"]
eval_dl_params['shuffle'] = False
data_layer = nemo_asr.AudioToTextDataLayer(
    manifest_filepath=','.join(EVAL_DATASETS),
    sample_rate=config.sample_rate,
    labels=config.labels,
    batch_size=32,
    **eval_dl_params,
)

[NeMo I 2020-03-09 10:19:20 collections:138] Dataset loaded with 278627 files totalling 948.67 hours
[NeMo I 2020-03-09 10:19:20 collections:139] 2614 files were filtered totalling 12.38 hours


In [171]:
data_preprocessor = nemo_asr.AudioToMelSpectrogramPreprocessor(
    sample_rate=config.sample_rate, **config.AudioToMelSpectrogramPreprocessor
)
jasper_encoder = nemo_asr.JasperEncoder(
    feat_in=config.AudioToMelSpectrogramPreprocessor["features"], **config.JasperEncoder
)
jasper_decoder = nemo_asr.JasperDecoderForCTC(
    feat_in=config.JasperEncoder["jasper"][-1]["filters"], num_classes=len(config.labels)
)
greedy_decoder = nemo_asr.GreedyCTCDecoder()

[NeMo I 2020-03-09 10:19:47 features:144] PADDING: 16
[NeMo I 2020-03-09 10:19:47 features:152] STFT using conv


In [172]:
audio_signal_e1, a_sig_length_e1, transcript_e1, transcript_len_e1 = data_layer()
processed_signal_e1, p_length_e1 = data_preprocessor(input_signal=audio_signal_e1, length=a_sig_length_e1)
encoded_e1, encoded_len_e1 = jasper_encoder(audio_signal=processed_signal_e1, length=p_length_e1)
log_probs_e1 = jasper_decoder(encoder_output=encoded_e1)
predictions_e1 = greedy_decoder(log_probs=log_probs_e1)

In [173]:
eval_tensors = [log_probs_e1, predictions_e1, transcript_e1, transcript_len_e1, encoded_len_e1, p_length_e1]
load_dir = '/home/stanislavv/data/nemo-quartznet-15x5'
evaluated_tensors = runner.infer(tensors=eval_tensors, checkpoint_dir=load_dir)

[NeMo I 2020-03-09 10:20:06 actions:1456] Restoring JasperEncoder from /home/stanislavv/data/nemo-quartznet-15x5/JasperEncoder-STEP-406556.pt
[NeMo I 2020-03-09 10:20:06 actions:1456] Restoring JasperDecoderForCTC from /home/stanislavv/data/nemo-quartznet-15x5/JasperDecoderForCTC-STEP-406556.pt
[NeMo I 2020-03-09 10:20:06 actions:729] Evaluating batch 0 out of 8708
[NeMo I 2020-03-09 10:25:19 actions:729] Evaluating batch 870 out of 8708
[NeMo I 2020-03-09 10:30:38 actions:729] Evaluating batch 1740 out of 8708
[NeMo I 2020-03-09 10:36:04 actions:729] Evaluating batch 2610 out of 8708
[NeMo I 2020-03-09 10:41:23 actions:729] Evaluating batch 3480 out of 8708
[NeMo I 2020-03-09 10:46:30 actions:729] Evaluating batch 4350 out of 8708
[NeMo I 2020-03-09 10:51:55 actions:729] Evaluating batch 5220 out of 8708
[NeMo I 2020-03-09 10:57:17 actions:729] Evaluating batch 6090 out of 8708
[NeMo I 2020-03-09 11:02:34 actions:729] Evaluating batch 6960 out of 8708
[NeMo I 2020-03-09 11:07:41 actio

In [174]:
references = post_process_transcripts(evaluated_tensors[2], evaluated_tensors[3], config.labels)
greedy_hypotheses = post_process_predictions(evaluated_tensors[1], config.labels)
word_error_rate(greedy_hypotheses, references)

0.004950049122295876

In [175]:
text = evaluated_tensors[2][0]
text_len = evaluated_tensors[3][0]
ctc_tokens = evaluated_tensors[1][0]
ctc_logprobs = evaluated_tensors[0][0]
ctc_len = evaluated_tensors[4][0]
mel_len = evaluated_tensors[-1][0]

In [176]:
text1 = text[0].numpy()
text_len1 = text_len[0].numpy().item()
ctc_tokens1 = ctc_tokens[0].numpy()
ctc_logprobs1 = ctc_logprobs[0].numpy()
ctc_len1 = ctc_len[0].numpy().item()
mel_len1 = mel_len[0].numpy().item()

In [177]:
text1 = list(text1[:text_len1])
ctc_tokens1 = list(ctc_tokens1[:ctc_len1])
ctc_logprobs1 = ctc_logprobs1[:ctc_len1]
len(text1), len(ctc_tokens1), ctc_logprobs1.shape

(274, 806, (806, 29))

In [178]:
class Processer:
    def __init__(self, labels):
        labels = labels + ['<BLANK>']
        self.space_id = labels.index(' ')
        self.blank_id = len(labels) - 1
        self.labels_map = dict([(i, labels[i]) for i in range(len(labels))])
    
    def bound_text(self, tokens):
        return [self.space_id] + tokens + [self.space_id]
    
    def bound_ctc(self, tokens, logprobs):
        tokens = [self.space_id, self.blank_id] + tokens + [self.blank_id, self.space_id]
        
        logprobs = np.lib.pad(logprobs, ((2, 2), (0, 0)), 'edge')

        def swap(col, a, b):
            x[col][a], x[col][b] = x[col][b], x[col][a]
        
        first_token, last_token = tokens[2], tokens[-3]
        swap(0, first_token, self.space_id)
        swap(1, first_token, self.blank_id)
        swap(-1, last_token, self.space_id)
        swap(-2, last_token, self.blank_id)

        return tokens, logprobs
    
    def merge(self, tokens):
        output_tokens = []
        output_cnts = []
        cnt = 0
        for i in range(len(tokens)):
            if i != 0 and (tokens[i - 1] != tokens[i]):
                output_tokens.append(tokens[i - 1])
                output_cnts.append(cnt)

                cnt = 0

            cnt += 1

        output_tokens.append(tokens[-1])
        output_cnts.append(cnt)
        
        assert sum(output_cnts) == len(tokens), f'SUM_CHECK {sum(output_cnts)} vs {len(tokens)}'

        return output_tokens, output_cnts
    
    def merge_with_blanks(self, tokens, cnts, logprobs=None):
        def choose_sep(l, r, a, b):
            # `tokens[l] == a and tokens[r] == b`.
            sum_a, sum_b = logprobs[l, a], logprobs[l + 1:r + 1, b].sum()
            best_sum, best_sep = sum_a + sum_b, 0
            for sep in range(1, r - l):
                sum_a += logprobs[l + sep, a]
                sum_b -= logprobs[l + sep, b]
                if sum_a + sum_b > best_sum:
                    best_sum, best_sep = sum_a + sum_b, sep

            return best_sep
        
        output_tokens = []
        output_durs = []
        blank_cnt = 0
        total_cnt = 0
        for token, cnt in zip(tokens, cnts):
            total_cnt += cnt
            if token == self.blank_id:
                blank_cnt += cnt
                continue
            
            output_tokens.append(token)
            
            if logprobs is None:
                # Half half.
                left_cnt = blank_cnt // 2
            else:
                # Clever sep choice based on sum of log probs.
                left_cnt = choose_sep(
                    l=total_cnt - cnt - blank_cnt - 1,
                    r=total_cnt - cnt,
                    a=output_tokens[-1],
                    b=token,
                )
            right_cnt = blank_cnt - left_cnt
            blank_cnt = 0
            
            if left_cnt:
                output_durs[-1] += left_cnt
            output_durs.append(cnt + right_cnt)
        
        output_durs[-1] += blank_cnt

        assert sum(output_durs) == sum(cnts), f'SUM_CHECK {sum(output_durs)} vs {sum(cnts)}'

        return output_tokens, output_durs
    
    def align(self, output_tokens, gt_text):
        def make_str(tokens):
            return ''.join(self.labels_map[c] for c in tokens)
        
        s = make_str(output_tokens)
        t = make_str(gt_text)
        alignmet = pairwise2.align.globalxx(s, t)[0]
        sa, ta, *_ = alignmet
        return sa, ta
    
    def generate(self, gt_text, alignment, durs):
        output_tokens = []
        output_cnts = []
        si, ti = 0, 0    
        for sc, tc in zip(*alignment):
            if sc == '-':
                output_tokens.append(self.blank_id)
                output_cnts.append(2 * durs[si])
                si += 1
            elif tc == '-':
                output_tokens.append(gt_text[ti])
                output_cnts.append(0)
                ti += 1
            else:
                output_tokens.append(gt_text[ti])
                output_cnts.append(2 * durs[si])
                si += 1
                ti += 1

        assert sum(output_cnts) == 2 * sum(durs)
        
        return output_tokens, output_cnts

    def __call__(self, text, ctc_tokens, ctc_logprobs, mel_len):
        # This adds +2 tokens.
        text = self.bound_text(text)
        # This add +4 tokens, 2 of them are blank.
        ctc_tokens, ctc_logprobs = self.bound_ctc(ctc_tokens, ctc_logprobs)

        ctc_tokens, ctc_cnts = self.merge(ctc_tokens)
        ctc_tokens, ctc_durs = self.merge_with_blanks(ctc_tokens, ctc_cnts, ctc_logprobs)
        
        alignment = self.align(text, ctc_tokens)
        tokens, cnts = self.generate(text, alignment, ctc_durs)
        tokens, durs = self.merge_with_blanks(tokens, cnts)
        assert tokens == text, 'EXACT_TOKENS_MATCH_CHECK'

        def adjust(start, direction, value):
            i = start
            while value != 0:
                dur = durs[i]
                
                if value < 0:
                    durs[i] = dur - value
                else:
                    durs[i] = max(dur - value, 0)
                
                value -= dur - durs[i]
                i += direction

        adjust(0, 1, 4)
        adjust(-1, -1, sum(durs) - mel_len)  # Including 4 suffix bound tokens.
        assert durs[0] >= 0, f'{durs[0]}'
        assert durs[-1] >= 0, f'{durs[-1]}'
        
        durs = np.array(durs, dtype=np.long)
        assert durs.shape[0] == len(text), f'LEN_CHECK {durs.shape[0]} vs {len(text)}'
        assert np.sum(durs) == mel_len, f'SUM_CHECK {np.sum(durs)} vs {mel_len}'

        return durs

processer = Processer(labels)
durs1 = processer(text1, ctc_tokens1, ctc_logprobs1, mel_len1)
durs1

array([ 0, 38,  2,  6,  4,  2,  2,  2, 12,  6,  4,  8,  4,  6,  6,  6,  6,
        4,  8, 12,  4,  8,  8, 32, 14,  4,  2,  2,  2, 10,  4,  6,  4,  2,
        8,  2,  2,  8,  2,  2,  2,  6,  6,  2,  4,  4,  2, 14,  6,  4,  2,
        6, 10,  2, 52, 16,  2,  4,  2,  6,  6,  6,  2,  8,  4,  2,  4,  6,
        6,  6,  6,  8,  4,  8,  8, 10,  2,  6,  2,  6,  4,  4,  4,  6,  8,
        2,  2,  8,  8,  4,  2,  2,  2, 52,  4,  4, 22, 20,  6,  4,  8,  2,
        2,  6,  4,  2,  6,  4,  2,  6,  4,  2,  4,  2,  4,  2,  2,  2,  2,
        4,  4,  4,  2,  2,  2,  2,  6,  2,  6,  8,  8,  4, 12,  2,  4, 24,
        2,  2,  2,  2,  6,  6,  2,  2,  2,  6,  2,  4,  2,  4,  4,  4,  4,
        4,  6,  6,  4,  6,  2,  2,  8,  4,  6,  2,  4,  8,  4,  4,  2,  8,
        6,  6,  4,  4, 14, 42, 10,  4,  2,  4,  4,  8,  2,  2,  2,  6, 12,
        6,  2,  4,  2, 14,  4,  4,  2, 12,  2,  2,  2,  8,  4,  2,  6,  6,
       10,  4,  4,  4, 14,  2,  2,  8,  4,  2,  2,  2,  4,  6,  8,  4,  4,
        6,  6,  2,  4,  6

In [179]:
f = lambda: processer(text1, ctc_tokens1, ctc_logprobs1, mel_len1)
n, m = 10000, 280000
(timeit.timeit(f, number=n) / n) * m

1658.5277968551964

In [180]:
# NOTE: This is going pretty fast.
durs_dir = pathlib.Path('/home/stanislavv/data/nemo-librispeech/durs')
durs_dir.mkdir(exist_ok=True)
k = -1

for batch in tqdm.tqdm(zip(*evaluated_tensors), total=8789):
    text = batch[2].numpy()
    text_len = batch[3].numpy()
    ctc_tokens = batch[1].numpy()
    ctc_logprobs = batch[0].numpy()
    ctc_len = batch[4].numpy()
    mel_len = batch[-1].numpy()

    for text1, text_len1, ctc_tokens1, ctc_logprobs1, ctc_len1, mel_len1 in zip(
        text, text_len, ctc_tokens, ctc_logprobs, ctc_len, mel_len
    ):
        text1 = list(text1[:text_len1])
        ctc_tokens1 = list(ctc_tokens1[:ctc_len1])
        ctc_logprobs1 = ctc_logprobs1[:ctc_len1]
        mel_len1 = mel_len1.item()

        durs1 = processer(text1, ctc_tokens1, ctc_logprobs1, mel_len1)

        k += 1
        np.save(durs_dir / f'{k}.npy', durs1, allow_pickle=False)





  0%|          | 0/8789 [00:00<?, ?it/s]



  0%|          | 1/8789 [00:00<20:36,  7.10it/s]



  0%|          | 2/8789 [00:00<19:15,  7.60it/s]



  0%|          | 3/8789 [00:00<19:36,  7.47it/s]



  0%|          | 4/8789 [00:00<19:26,  7.53it/s]



  0%|          | 5/8789 [00:00<19:55,  7.35it/s]



  0%|          | 6/8789 [00:00<19:31,  7.50it/s]



  0%|          | 7/8789 [00:00<19:48,  7.39it/s]



  0%|          | 8/8789 [00:01<19:19,  7.57it/s]



  0%|          | 9/8789 [00:01<20:49,  7.03it/s]



  0%|          | 10/8789 [00:01<21:04,  6.94it/s]



  0%|          | 11/8789 [00:01<22:01,  6.64it/s]



  0%|          | 12/8789 [00:01<21:14,  6.89it/s]



  0%|          | 13/8789 [00:01<20:50,  7.02it/s]



  0%|          | 14/8789 [00:01<19:55,  7.34it/s]



  0%|          | 15/8789 [00:02<19:13,  7.61it/s]



  0%|          | 16/8789 [00:02<19:35,  7.46it/s]



  0%|          | 17/8789 [00:02<20:34,  7.11it/s]



  0%|          | 18/8789 [00:02<20:01,  7.30it/s]



  0%|  

  1%|▏         | 127/8789 [00:17<18:02,  8.00it/s]



  1%|▏         | 128/8789 [00:17<18:20,  7.87it/s]



  1%|▏         | 129/8789 [00:17<19:08,  7.54it/s]



  1%|▏         | 130/8789 [00:17<20:01,  7.21it/s]



  1%|▏         | 131/8789 [00:17<20:03,  7.19it/s]



  2%|▏         | 132/8789 [00:18<20:48,  6.93it/s]



  2%|▏         | 133/8789 [00:18<21:16,  6.78it/s]



  2%|▏         | 134/8789 [00:18<21:41,  6.65it/s]



  2%|▏         | 135/8789 [00:18<21:03,  6.85it/s]



  2%|▏         | 136/8789 [00:18<21:16,  6.78it/s]



  2%|▏         | 137/8789 [00:18<21:34,  6.68it/s]



  2%|▏         | 138/8789 [00:18<20:29,  7.04it/s]



  2%|▏         | 139/8789 [00:18<19:11,  7.51it/s]



  2%|▏         | 140/8789 [00:19<18:42,  7.71it/s]



  2%|▏         | 141/8789 [00:19<18:54,  7.62it/s]



  2%|▏         | 142/8789 [00:19<19:11,  7.51it/s]



  2%|▏         | 143/8789 [00:19<19:58,  7.21it/s]



  2%|▏         | 144/8789 [00:19<21:00,  6.86it/s]



  2%|▏         | 145/8789 [0

  3%|▎         | 255/8789 [00:34<22:24,  6.35it/s]



  3%|▎         | 256/8789 [00:34<21:06,  6.74it/s]



  3%|▎         | 257/8789 [00:34<19:53,  7.15it/s]



  3%|▎         | 258/8789 [00:34<18:40,  7.61it/s]



  3%|▎         | 259/8789 [00:35<18:24,  7.72it/s]



  3%|▎         | 260/8789 [00:35<17:56,  7.92it/s]



  3%|▎         | 261/8789 [00:35<18:07,  7.84it/s]



  3%|▎         | 262/8789 [00:35<18:21,  7.74it/s]



  3%|▎         | 263/8789 [00:35<17:58,  7.91it/s]



  3%|▎         | 264/8789 [00:35<17:53,  7.94it/s]



  3%|▎         | 265/8789 [00:35<18:18,  7.76it/s]



  3%|▎         | 267/8789 [00:35<17:34,  8.08it/s]



  3%|▎         | 268/8789 [00:36<17:23,  8.16it/s]



  3%|▎         | 269/8789 [00:36<18:10,  7.81it/s]



  3%|▎         | 270/8789 [00:36<18:18,  7.76it/s]



  3%|▎         | 271/8789 [00:36<18:04,  7.86it/s]



  3%|▎         | 272/8789 [00:36<17:53,  7.93it/s]



  3%|▎         | 273/8789 [00:36<17:07,  8.29it/s]



  3%|▎         | 274/8789 [0

  4%|▍         | 385/8789 [00:51<18:13,  7.68it/s]



  4%|▍         | 386/8789 [00:51<17:39,  7.93it/s]



  4%|▍         | 387/8789 [00:51<17:13,  8.13it/s]



  4%|▍         | 388/8789 [00:51<16:46,  8.34it/s]



  4%|▍         | 389/8789 [00:51<16:45,  8.36it/s]



  4%|▍         | 390/8789 [00:51<17:15,  8.11it/s]



  4%|▍         | 391/8789 [00:52<17:37,  7.95it/s]



  4%|▍         | 392/8789 [00:52<17:04,  8.20it/s]



  4%|▍         | 393/8789 [00:52<16:40,  8.39it/s]



  4%|▍         | 394/8789 [00:52<16:43,  8.37it/s]



  4%|▍         | 395/8789 [00:52<17:10,  8.15it/s]



  5%|▍         | 396/8789 [00:52<17:41,  7.91it/s]



  5%|▍         | 397/8789 [00:52<16:56,  8.25it/s]



  5%|▍         | 398/8789 [00:52<18:04,  7.74it/s]



  5%|▍         | 399/8789 [00:53<18:42,  7.47it/s]



  5%|▍         | 400/8789 [00:53<18:34,  7.53it/s]



  5%|▍         | 401/8789 [00:53<18:35,  7.52it/s]



  5%|▍         | 402/8789 [00:53<19:15,  7.26it/s]



  5%|▍         | 403/8789 [0

  6%|▌         | 513/8789 [01:08<20:22,  6.77it/s]



  6%|▌         | 514/8789 [01:08<20:36,  6.69it/s]



  6%|▌         | 515/8789 [01:08<19:31,  7.06it/s]



  6%|▌         | 516/8789 [01:08<18:52,  7.30it/s]



  6%|▌         | 517/8789 [01:08<19:15,  7.16it/s]



  6%|▌         | 518/8789 [01:08<19:20,  7.13it/s]



  6%|▌         | 519/8789 [01:08<19:38,  7.02it/s]



  6%|▌         | 520/8789 [01:09<19:35,  7.04it/s]



  6%|▌         | 521/8789 [01:09<19:41,  7.00it/s]



  6%|▌         | 522/8789 [01:09<19:54,  6.92it/s]



  6%|▌         | 523/8789 [01:09<21:18,  6.47it/s]



  6%|▌         | 524/8789 [01:09<20:36,  6.68it/s]



  6%|▌         | 525/8789 [01:09<20:51,  6.61it/s]



  6%|▌         | 526/8789 [01:10<20:33,  6.70it/s]



  6%|▌         | 527/8789 [01:10<20:25,  6.74it/s]



  6%|▌         | 528/8789 [01:10<19:11,  7.18it/s]



  6%|▌         | 529/8789 [01:10<18:52,  7.30it/s]



  6%|▌         | 530/8789 [01:10<17:56,  7.67it/s]



  6%|▌         | 531/8789 [0

...

 92%|█████████▏| 8097/8789 [17:22<01:27,  7.95it/s]



 92%|█████████▏| 8098/8789 [17:22<01:27,  7.90it/s]



 92%|█████████▏| 8099/8789 [17:22<01:35,  7.20it/s]



 92%|█████████▏| 8100/8789 [17:22<01:32,  7.41it/s]



 92%|█████████▏| 8101/8789 [17:22<01:31,  7.49it/s]



 92%|█████████▏| 8102/8789 [17:22<01:28,  7.77it/s]



 92%|█████████▏| 8103/8789 [17:23<01:22,  8.32it/s]



 92%|█████████▏| 8105/8789 [17:23<01:17,  8.85it/s]



 92%|█████████▏| 8106/8789 [17:23<01:16,  8.98it/s]



 92%|█████████▏| 8107/8789 [17:23<01:19,  8.58it/s]



 92%|█████████▏| 8108/8789 [17:23<01:23,  8.13it/s]



 92%|█████████▏| 8109/8789 [17:23<01:24,  8.02it/s]



 92%|█████████▏| 8110/8789 [17:23<01:22,  8.26it/s]



 92%|█████████▏| 8111/8789 [17:24<01:22,  8.26it/s]



 92%|█████████▏| 8112/8789 [17:24<01:20,  8.40it/s]



 92%|█████████▏| 8113/8789 [17:24<01:20,  8.35it/s]



 92%|█████████▏| 8114/8789 [17:24<01:18,  8.61it/s]



 92%|█████████▏| 8115/8789 [17:24<01:17,  8.66it/s]



 92%|█████

 94%|█████████▎| 8229/8789 [17:38<01:08,  8.23it/s]



 94%|█████████▎| 8230/8789 [17:38<01:10,  7.94it/s]



 94%|█████████▎| 8231/8789 [17:38<01:08,  8.13it/s]



 94%|█████████▎| 8232/8789 [17:38<01:10,  7.92it/s]



 94%|█████████▎| 8233/8789 [17:39<01:12,  7.62it/s]



 94%|█████████▎| 8234/8789 [17:39<01:15,  7.35it/s]



 94%|█████████▎| 8235/8789 [17:39<01:18,  7.10it/s]



 94%|█████████▎| 8236/8789 [17:39<01:18,  7.03it/s]



 94%|█████████▎| 8237/8789 [17:39<01:18,  7.05it/s]



 94%|█████████▎| 8238/8789 [17:39<01:13,  7.46it/s]



 94%|█████████▍| 8240/8789 [17:40<01:08,  7.99it/s]



 94%|█████████▍| 8241/8789 [17:40<01:07,  8.08it/s]



 94%|█████████▍| 8242/8789 [17:40<01:06,  8.28it/s]



 94%|█████████▍| 8243/8789 [17:40<01:10,  7.79it/s]



 94%|█████████▍| 8244/8789 [17:40<01:13,  7.37it/s]



 94%|█████████▍| 8245/8789 [17:40<01:16,  7.15it/s]



 94%|█████████▍| 8246/8789 [17:40<01:16,  7.12it/s]



 94%|█████████▍| 8247/8789 [17:40<01:13,  7.37it/s]



 94%|█████

 95%|█████████▌| 8372/8789 [17:55<00:46,  8.92it/s]



 95%|█████████▌| 8373/8789 [17:55<00:45,  9.09it/s]



 95%|█████████▌| 8374/8789 [17:56<00:45,  9.06it/s]



 95%|█████████▌| 8375/8789 [17:56<00:47,  8.74it/s]



 95%|█████████▌| 8376/8789 [17:56<00:47,  8.78it/s]



 95%|█████████▌| 8377/8789 [17:56<00:48,  8.48it/s]



 95%|█████████▌| 8378/8789 [17:56<00:52,  7.82it/s]



 95%|█████████▌| 8379/8789 [17:56<00:54,  7.58it/s]



 95%|█████████▌| 8380/8789 [17:56<00:54,  7.54it/s]



 95%|█████████▌| 8381/8789 [17:56<00:52,  7.74it/s]



 95%|█████████▌| 8382/8789 [17:57<00:50,  8.10it/s]



 95%|█████████▌| 8383/8789 [17:57<00:47,  8.58it/s]



 95%|█████████▌| 8384/8789 [17:57<00:47,  8.50it/s]



 95%|█████████▌| 8385/8789 [17:57<00:48,  8.40it/s]



 95%|█████████▌| 8386/8789 [17:57<00:49,  8.06it/s]



 95%|█████████▌| 8387/8789 [17:57<00:52,  7.64it/s]



 95%|█████████▌| 8388/8789 [17:57<00:53,  7.50it/s]



 95%|█████████▌| 8389/8789 [17:57<00:53,  7.43it/s]



 95%|█████

 97%|█████████▋| 8500/8789 [18:12<00:36,  7.92it/s]



 97%|█████████▋| 8501/8789 [18:12<00:36,  7.86it/s]



 97%|█████████▋| 8502/8789 [18:12<00:38,  7.46it/s]



 97%|█████████▋| 8503/8789 [18:12<00:39,  7.24it/s]



 97%|█████████▋| 8504/8789 [18:12<00:41,  6.95it/s]



 97%|█████████▋| 8505/8789 [18:12<00:41,  6.87it/s]



 97%|█████████▋| 8506/8789 [18:13<00:40,  6.94it/s]



 97%|█████████▋| 8507/8789 [18:13<00:40,  7.00it/s]



 97%|█████████▋| 8508/8789 [18:13<00:38,  7.24it/s]



 97%|█████████▋| 8509/8789 [18:13<00:38,  7.35it/s]



 97%|█████████▋| 8510/8789 [18:13<00:37,  7.36it/s]



 97%|█████████▋| 8511/8789 [18:13<00:36,  7.58it/s]



 97%|█████████▋| 8512/8789 [18:13<00:37,  7.36it/s]



 97%|█████████▋| 8513/8789 [18:14<00:36,  7.47it/s]



 97%|█████████▋| 8514/8789 [18:14<00:37,  7.42it/s]



 97%|█████████▋| 8515/8789 [18:14<00:35,  7.73it/s]



 97%|█████████▋| 8516/8789 [18:14<00:35,  7.64it/s]



 97%|█████████▋| 8517/8789 [18:14<00:41,  6.62it/s]



 97%|█████

 98%|█████████▊| 8632/8789 [18:29<00:16,  9.45it/s]



 98%|█████████▊| 8633/8789 [18:29<00:18,  8.61it/s]



 98%|█████████▊| 8634/8789 [18:29<00:18,  8.25it/s]



 98%|█████████▊| 8635/8789 [18:29<00:18,  8.16it/s]



 98%|█████████▊| 8636/8789 [18:29<00:18,  8.17it/s]



 98%|█████████▊| 8637/8789 [18:29<00:17,  8.51it/s]



 98%|█████████▊| 8639/8789 [18:30<00:16,  9.05it/s]



 98%|█████████▊| 8640/8789 [18:30<00:16,  9.23it/s]



 98%|█████████▊| 8642/8789 [18:30<00:15,  9.75it/s]



 98%|█████████▊| 8643/8789 [18:30<00:16,  9.07it/s]



 98%|█████████▊| 8644/8789 [18:30<00:17,  8.43it/s]



 98%|█████████▊| 8645/8789 [18:30<00:18,  7.87it/s]



 98%|█████████▊| 8646/8789 [18:30<00:18,  7.83it/s]



 98%|█████████▊| 8647/8789 [18:30<00:17,  7.94it/s]



 98%|█████████▊| 8648/8789 [18:31<00:18,  7.74it/s]



 98%|█████████▊| 8649/8789 [18:31<00:18,  7.64it/s]



 98%|█████████▊| 8650/8789 [18:31<00:18,  7.59it/s]



 98%|█████████▊| 8651/8789 [18:31<00:19,  7.22it/s]



 98%|█████